## Plot the final water level maps

**Previous code:**
create_final_water_level_maps.ipynb

**Aim:**
Output statistical maps of the derived water level maps
- min, max, mean, stdev monthly maps (for supplementary paper)
- min, max, mean, stdev across all timesteps (main paper)
- number of days with water level above ground level


**Inputs**
- final water level map

**Outputs**
- Figures from the paper: 10, 11, 12

**Next**
- Final validation with altimetry data

**Notes**
- The final criteria used for applying the rainfed interpolation is: pixel-wise pvalue < 0.15
- Otherwise, the linear interpolation is used. 
- This covers the flood bank regions, and regions where there are negative pixel wise correlations. 
- About 10% of the region has little (<0.1) to no correlation (<=0) with the net water input.
- About 75% of pixels are to some degree (correlation >0.3) rainfed


In [ ]:
import sys
import pandas as pd
import xarray as xr
import numpy as np
xr.set_options(cmap_sequential='jet')
import matplotlib.pyplot as plt
import warnings
import netCDF4
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go
from affine import Affine
import cartopy.crs as ccrs
import nco
import xesmf as xe
from calendar import monthrange
import pickle as pkl
from datetime import date, datetime, timedelta
import math
import seaborn as sns
from scipy import stats
from IPython.display import display

plt.rcParams['figure.dpi'] = 700

%xmode Minimal

In [ ]:
### Declare data and output directories here: 
#ALOS_OUT = 
#images = 

In [ ]:
## Water level datasets

# rainfed
ds_r = xr.open_dataset(ALOS_OUT + 'HH_modelled_water_level_ts.nc')

# linear
ds_l = xr.open_dataset(ALOS_OUT + 'HH_linear_water_level_ts.nc')

# final water level
wl_final = xr.open_dataset(ALOS_OUT + 'WL_daily_final.nc')

# land type map
lt_map = xr.open_dataset(ALOS_OUT + 'landtype_100m.nc')

# pixel wise correlation slope data
pw_slopes = xr.open_dataset(ALOS_OUT + 'slopes_da_HH_new.nc')
test = pw_slopes.to_array().to_dataset(name='slope')
pw_slopes = test['slope'][0][0].to_dataset(name='slope')
pw_slopes = pw_slopes.where(lt_map['type'].isin([4,5]))
pw_slopes = pw_slopes.drop('variable')

# 2. correlations
pw_corrs = xr.open_dataset(ALOS_OUT + 'corrs_da_HH_new.nc')
test = pw_corrs.to_array().to_dataset(name='correlation')
pw_corrs = test['correlation'][0][0].to_dataset(name='correlation')
pw_corrs = pw_corrs.where(lt_map['type'].isin([4,5]))
pw_corrs = pw_corrs.drop('variable')

# 3. p-values
pw_pvals = xr.open_dataset(ALOS_OUT + 'pvals_da_HH_new.nc')
test = pw_pvals.to_array().to_dataset(name='pvalue')
pw_pvals = test['pvalue'][0][0].to_dataset(name='pvalue')
pw_pvals = pw_pvals.where(lt_map['type'].isin([4,5]))
pw_pvals = pw_pvals.drop('variable')

# 4. standard errors
pw_stderrs = xr.open_dataset(ALOS_OUT + 'stderrs_da_HH_new.nc')
test = pw_stderrs.to_array().to_dataset(name='stderr')
pw_stderrs = test['stderr'][0][0].to_dataset(name='stderr')
pw_stderrs = pw_stderrs.where(lt_map['type'].isin([4,5]))
pw_stderrs = pw_stderrs.drop('variable')

# original radar data
HH_db_100m = xr.open_dataset(ALOS_OUT + 'HH_db_100m_area2.nc')

### Function declarations

In [ ]:
import matplotlib.colors as mcolors
from matplotlib.colors import Normalize
import matplotlib as mpl
#from matplotlib.colors import MidpointNormalize

def plot_maps(ds, var_name, vmin, vmax):
    #mid = -vmin/(vmax - vmin)
    # Define the colours for the colourmap
    #colours = [(0, 'red'), (mid, 'white'), (1, 'blue')]
    
    #midnorm = MidpointNormalize(midpoint=0, vmin=vmin, vmax=vmax)
    
    #cmap = mpl.colormaps['RdBu']
    midpoint = 1 - vmax / (vmax + abs(vmin))
    print (midpoint)
    start = 0.12
    cmap_new = shiftedColorMap(matplotlib.cm.RdBu, start=start, midpoint= midpoint, stop=1, name='shiftedcmap')
    
    #sns.set(font_scale=1.2, rc={'axes.facecolor':'#D1D3D4'}, style='ticks')
    sns.set(font_scale=1.3, rc={'axes.facecolor':'white'}, style='ticks')



    #cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', colours, N=256)

    # register the colourmap
    #plt.register_cmap(cmap=cmap)
    
    # formatting the time for better subplot titles
    time_formatted = ds['time'].dt.strftime('%b %Y')
    
    #fig, ax = plt.subplots(subplot_kw={'title':time_formatted})
    
    fig, ax = plt.subplots(nrows=4, ncols=5, figsize = (18,16))
    ax = ax.ravel()

    # adding more space between rows
    plt.subplots_adjust(hspace=0.5)
    #plt.title('Maximum monthly water level (cm)')
    
    for i, subplot in enumerate(ax):
        ds[var_name].isel(time=i).plot(x='lon',y='lat',ax=subplot,vmin=vmin,vmax=vmax, \
                                                  cmap=cmap_new, add_colorbar=False)
        subplot.set_xlabel('')
        subplot.set_ylabel('')
        subplot.set_title(time_formatted.isel(time=i).item())
    
    cax = fig.add_axes([0.92, 0.1, 0.02, 0.8])
    #plt.title('test')
    # normalising the colourbar so that the white value is at 0 water level
    norm = Normalize(vmin=vmin, vmax=vmax)
    print (norm)
    cbar = plt.colorbar(subplot.collections[0], cax=cax)
    cbar.set_label(var_name, size=18)
        #if (i+1) % 5 == 0:
        #    cbar = plt.colorbar(subplot.collections[0], ax=ax, shrink=1)
    
    # original version that didn't use subplots, but the layout wasn't as good
    #ds[var_name][:,::5,::5].plot(x='lon',y='lat',col='time',col_wrap=5,\
    #                                  vmin=vmin,vmax=vmax, ax=ax, cmap='custom_cmap', )


In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    #### This function was copied from: https://stackoverflow.com/questions/7404116/defining-the-midpoint-of-a-colormap-in-matplotlib
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
def plot_maps_overall(ds, var_name, vmin, vmax, cmap):
    # plotting the mean/min/max/stdev etc over all data
    sns.set(font_scale=1.3, rc={'axes.facecolor':'white'}, style='ticks')
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize = (10,7))


    ds[var_name].plot(x='lon',y='lat',ax=ax,vmin=vmin,vmax=vmax, \
                                              cmap=cmap, add_colorbar=True)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title('')
    
    


### Max, min, mean, std dev across all 20 image months
 - final paper images 

In [ ]:
%%time
# sum of number of days where the water level is above 0 cm 
sum_0 = wl_final.where(wl_final['water_level']>0).count(dim='time', keep_attrs=True)

In [ ]:
sum_0.to_netcdf(ALOS_OUT + 'sum_0.nc')

In [ ]:
%%time
# sum of number of days where the water level is above -5 cm 
sum_5 = wl_final.where(wl_final['water_level']>-5).count(dim='time', keep_attrs=True)

In [ ]:
sum_5.to_netcdf(ALOS_OUT + 'sum_5.nc')

In [ ]:
## Quickstart option if above code was previously run
sum_0 = xr.open_dataset(ALOS_OUT + 'sum_0.nc')
sum_5 = xr.open_dataset(ALOS_OUT + 'sum_5.nc')

In [ ]:
sum_0 = sum_0.where(lt_map['type'].isin([4,5])).drop('band')
sum_5 = sum_5.where(lt_map['type'].isin([4,5])).drop('band')

### calculating the percentage of days that the water level is above the peat surface

In [ ]:
# -8db is the minimum backscatter over peat pixels
ds0 = sum_0.where(HH_db_100m['db'][0]>-8)

# Calculating the percentage of days with the water level above 0 cm for palm and hardwood swamp seperately - 560 days in current time series
ds0['water_level'] = ds0['water_level']*100/560
sum_0_4 = ds0['water_level'].copy().where(lt_map['type'].isin([4])).to_dataset(name = 'palm swamp').drop('band')
sum_0_5 = ds0['water_level'].copy().where(lt_map['type'].isin([5])).to_dataset(name = 'hardwood swamp').drop('band')

ps_df = sum_0_4.to_dataframe().reset_index(drop=True)
hws_df = sum_0_5.to_dataframe().reset_index(drop=True)
peat_df = pd.concat([ps_df, hws_df]).melt(value_vars = ['palm swamp', 'hardwood swamp'])


In [ ]:
var_name = '% of days with water level above -5cm (below ground)'
ds5 = sum_5.where(HH_db_100m['db'][0]>-8)

# masking for pixels with below 3 % of days
ds5 = ds5.where(ds5['water_level']>3)
ds5[var_name] = ds5['water_level']*100/560
plot_maps_overall(ds5, var_name,60, 100,'viridis_r')

In [ ]:
var_name = '% of days with water level above the peat surface'
ds0 = sum_0.where(HH_db_100m['db'][0]>-8)
ds0 = ds0.where(ds0['water_level']>3)
ds0[var_name] = ds0['water_level']*100/560
plot_maps_overall(ds0, var_name,0, 100,'viridis_r')
plt.savefig(images + 'WL0_days.png')

In [ ]:
ds100 = ds0.where(ds0['% of days with water level above the peat surface']==100)
plot_maps_overall(ds100, var_name,0, 100,'viridis_r')

### QUICKSTART

In [ ]:
WL_mean = xr.open_dataset(ALOS_OUT + 'WL_mean.nc')
WL_min = xr.open_dataset(ALOS_OUT + 'WL_min.nc')
WL_max = xr.open_dataset(ALOS_OUT + 'WL_max.nc')
WL_stdev = xr.open_dataset(ALOS_OUT + 'WL_stdev.nc')

In [ ]:
WL_amplitude = WL_max['water_level'] - WL_min['water_level']
var_name = 'Water level amplitude (cm)'
ds = WL_amplitude.to_dataset(name = var_name)
ds[var_name].plot.hist(xlim=(5,70), bins=300);
plt.title('')

# setting the y axis format to include a comma between the thousands for the count
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))

plt.annotate('Mean value: ' + str(np.round(WL_amplitude.mean().values,1)) + ' cm', (30,200000))
plt.annotate('$\pm$'+ str(np.round(WL_amplitude.std().values,1)) + ' cm to 1$\sigma$', (30,170000))
plt.savefig(images + 'WL_amplitude_hist.png', bbox_inches='tight')

In [ ]:
# flooded pixels
ds_f = ds.where(pw_pvals['pvalue']>=0.15)
ds_f[var_name].plot.hist(xlim=(5,70), bins=300);
plt.title('')
plt.annotate('Mean value: ' + str(np.round(ds_f[var_name].mean().values,1)) + ' cm', (30,30000))
plt.annotate('$\pm$'+ str(np.round(ds_f[var_name].std().values,1)) + ' cm to 1$\sigma$', (30,26000))

In [ ]:
# rainfed pixels
ds_r = ds.where(pw_pvals['pvalue']<0.05)
ds_r[var_name].plot.hist(xlim=(5,70), bins=300);
plt.title('')
plt.annotate('Mean value: ' + str(np.round(ds_r[var_name].mean().values,1)) + ' cm', (30,75000))
plt.annotate('$\pm$'+ str(np.round(ds_r[var_name].std().values,1)) + ' cm to 1$\sigma$', (30,65000))

In [ ]:
var_name = 'Water level amplitude (cm)'
ds = WL_amplitude.to_dataset(name = var_name)
ds = ds.where(HH_db_100m['db'][0]>-8)
#colors = [(0.9,0.6,0.4), (0.4,0.6,0.8)]  # Brown and Blue
#positions = [0, 1]
#cmap = mcolors.LinearSegmentedColormap.from_list('terrain_brown_blue', list(zip(positions, colors)))
vmin=10
vmax=40
midpoint = 1 - vmax / (vmax + abs(vmin))
print (midpoint)
start = 0.0
cmap_new = shiftedColorMap(matplotlib.cm.YlGnBu, start=start, midpoint= midpoint, stop=1, name='shiftedcmap')
plot_maps_overall(ds, var_name, vmin, vmax,cmap_new)
plt.savefig(images + 'WL_amplitude.png')

In [ ]:
var_name = 'Mean water level (cm)'
ds = WL_mean.to_array().to_dataset(name = var_name)
ds = ds.where(HH_db_100m['db'][0]>-8)
#colors = [(0.9,0.6,0.4), (0.4,0.6,0.8)]  # Brown and Blue
#positions = [0, 1]
#cmap = mcolors.LinearSegmentedColormap.from_list('terrain_brown_blue', list(zip(positions, colors)))
vmin=-10
vmax=50
midpoint = 1 - vmax / (vmax + abs(vmin))
print (midpoint)
start = 0.0
cmap_new = shiftedColorMap(matplotlib.cm.YlGnBu, start=start, midpoint= midpoint, stop=1, name='shiftedcmap')
plot_maps_overall(ds, var_name, vmin, vmax,cmap_new)

In [ ]:
var_name = 'Mean water level (cm)'
ds = WL_mean.to_array().to_dataset(name = var_name)
ds = ds.where(HH_db_100m['db'][0]>-8)
#colors = [(0.9,0.6,0.4), (0.4,0.6,0.8)]  # Brown and Blue
#positions = [0, 1]
#cmap = mcolors.LinearSegmentedColormap.from_list('terrain_brown_blue', list(zip(positions, colors)))
vmin=-10
vmax=50
midpoint = 1 - vmax / (vmax + abs(vmin))
print (midpoint)
start = 0.0
cmap_new = shiftedColorMap(matplotlib.cm.YlGnBu, start=start, midpoint= midpoint, stop=1, name='shiftedcmap')
plot_maps_overall(ds, var_name, vmin, vmax,cmap_new)

In [ ]:
var_name = 'Maximum water level (cm)'
ds = WL_max.to_array().to_dataset(name = var_name)
ds = ds.where(HH_db_100m['db'][0]>-8)
#colors = [(0.9,0.6,0.4), (0.4,0.6,0.8)]  # Brown and Blue
#positions = [0, 1]
#cmap = mcolors.LinearSegmentedColormap.from_list('terrain_brown_blue', list(zip(positions, colors)))
vmin=-10
vmax=50
midpoint = 1 - vmax / (vmax + abs(vmin))
print (midpoint)
start = 0.0 
cmap_new = shiftedColorMap(matplotlib.cm.YlGnBu, start=start, midpoint= midpoint, stop=1, name='shiftedcmap')
plot_maps_overall(ds, var_name, vmin, vmax,cmap_new)

In [ ]:
var_name = 'Minimum water level (cm)'
ds = WL_min.to_array().to_dataset(name = var_name)
ds = ds.where(HH_db_100m['db'][0]>-8)
#colors = [(0.9,0.6,0.4), (0.4,0.6,0.8)]  # Brown and Blue
#positions = [0, 1]
#cmap = mcolors.LinearSegmentedColormap.from_list('terrain_brown_blue', list(zip(positions, colors)))
vmin=-10
vmax=50
midpoint = 1 - vmax / (vmax + abs(vmin))
print (midpoint)
start = 0.0 
cmap_new = shiftedColorMap(matplotlib.cm.YlGnBu, start=start, midpoint= midpoint, stop=1, name='shiftedcmap')
plot_maps_overall(ds, var_name, vmin, vmax,cmap_new)

In [ ]:
ds = WL_stdev.to_array().to_dataset(name = 'standard deviation (cm)')
ds = ds.where(HH_db_100m['db'][0]>-8)
plot_maps_overall(ds, 'standard deviation (cm)', 0, 15,'RdBu_r')

In [ ]:
var_name = 'Standard deviation of the water level (cm)'
ds = WL_stdev.to_array().to_dataset(name = var_name)
ds = ds.where(HH_db_100m['db'][0]>-8)
#colors = [(0.9,0.6,0.4), (0.4,0.6,0.8)]  # Brown and Blue
#positions = [0, 1]
#cmap = mcolors.LinearSegmentedColormap.from_list('terrain_brown_blue', list(zip(positions, colors)))
vmin=0
vmax=15
midpoint = 1 - vmax / (vmax + abs(vmin))
print (midpoint)
start = -1.2
cmap_new = shiftedColorMap(matplotlib.cm.viridis, start=start, midpoint= midpoint, stop=1.2, name='shiftedcmap')
plot_maps_overall(ds, var_name, vmin, vmax,cmap_new)

In [ ]:
# saving to netcdf
WL_mean.to_netcdf(ALOS_OUT + 'WL_mean.nc')
WL_min.to_netcdf(ALOS_OUT + 'WL_min.nc')
WL_max.to_netcdf(ALOS_OUT + 'WL_max.nc')
WL_stdev.to_netcdf(ALOS_OUT + 'WL_stdev.nc')

### Max, min, mean and standard deviation over each month in the water level sequence
- final paper images (supplementary)

In [ ]:
%%time
# resampling to monthly
print ('resampling mean...')
WL_monthly_mean = wl_final['water_level'].resample(time='1M').mean()

print ('resampling max...')
WL_monthly_max = wl_final['water_level'].resample(time='1M').max()

print ('resampling min...')
WL_monthly_min = wl_final['water_level'].resample(time='1M').min()

print ('resampling std dev...')
WL_monthly_stdev = wl_final['water_level'].resample(time='1M').std()


In [ ]:
# save to netcdf
WL_monthly_mean.to_netcdf(ALOS_OUT + 'WL_monthly_mean.nc')
WL_monthly_min.to_netcdf(ALOS_OUT + 'WL_monthly_min.nc')
WL_monthly_max.to_netcdf(ALOS_OUT + 'WL_monthly_max.nc')
WL_monthly_stdev.to_netcdf(ALOS_OUT + 'WL_monthly_stdev.nc')

In [ ]:
%%time
# resampling the linear data to monthly - comparing this with the final resampled data
# perhaps best to do this for both the linear and rainfed data, before deciding on with p-value/correlation
# criteria to use
print ('resampling mean...')
WL_monthly_mean_l = ds_l['water_level'].resample(time='1M').mean()

print ('resampling max...')
WL_monthly_max_l = ds_l['water_level'].resample(time='1M').max()

print ('resampling min...')
WL_monthly_min_l = ds_l['water_level'].resample(time='1M').min()

print ('resampling std dev...')
WL_monthly_stdev_l = ds_l['water_level'].resample(time='1M').std()

In [ ]:
%%time
# save to netcdf
WL_monthly_mean_l.to_netcdf(ALOS_OUT + 'WL_monthly_mean_l.nc')
WL_monthly_min_l.to_netcdf(ALOS_OUT + 'WL_monthly_min_l.nc')
WL_monthly_max_l.to_netcdf(ALOS_OUT + 'WL_monthly_max_l.nc')
WL_monthly_stdev_l.to_netcdf(ALOS_OUT + 'WL_monthly_stdev_l.nc')